In [1]:
import pandas as pd
import numpy as np

import tarfile
import os
import csv

import librosa
import librosa.display

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
data, sampling_rate = librosa.load('./data/Healthy/a_0016.wav')
data, data.shape, sampling_rate

(array([-1.5258789e-04, -1.2207031e-04, -1.5258789e-04, ...,
         9.1552734e-05,  0.0000000e+00, -9.1552734e-05], dtype=float32),
 (90028,),
 22050)

In [3]:
for x in os.listdir('./data'):
    if os.path.isdir('./data/'+x):
        print(x, ": ", len(os.listdir('./data/'+x)))

Healthy :  161
Pathological :  166


In [4]:
d = {}
digit = ['Healthy', 'Pathological']
with open("Spoken_digit.csv", 'w') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(["File", "Label"])
    for x in digit:
        if os.path.isdir('./data/'+x):
            d[x] = os.listdir('./data/'+x)
            for name in os.listdir('./data/'+x):
                if os.path.isfile('./data/'+x+"/"+name):
                    csvwriter.writerow([x+'/'+name, x])

#shuffle 
df = pd.read_csv('Spoken_digit.csv')
df = df.sample(frac=1)
df.to_csv('Spoken_digit.csv', index = False)

In [5]:
sp = pd.read_csv("Spoken_digit.csv")

### MFCC

In [6]:
csvfile = open("feature_MFCC.csv", "w")
csvwriter = csv.writer(csvfile)
csvwriter.writerow(np.concatenate((['Label'], [i for i in range(1, 14)])))

37

In [7]:
def extract_MFCCfeatures(files):
    data, sr = librosa.load('./data/'+files.File)
#     mfccs = np.mean(librosa.feature.mfcc(y = data, sr = sr, n_mfcc = 13).T, axis = 0)
#     stft = np.abs(librosa.stft(data))
#     chroma = np.mean(librosa.feature.chroma_stft(S = stft, sr = sr).T, axis = 0)
#     mel = np.mean(librosa.feature.melspectrogram(data, sr).T, axis = 0)
#     contrast = np.mean(librosa.feature.spectral_contrast(S = stft, sr = sr).T, axis = 0)
#     tonnetz = np.mean(librosa.feature.tonnetz(y = librosa.effects.harmonic(data), sr = sr).T, axis = 0)
#     row =  np.concatenate((mfccs, chroma, mel, contrast, tonnetz), axis = 0).astype('float32')
#     csvwriter.writerow(np.concatenate(([digit.index(files.Label)], row)))
    mean_mfccs = np.mean(librosa.feature.mfcc(y = data, sr = sr, n_mfcc = 13).T, axis = 0)
    csvwriter.writerow(np.concatenate(([digit.index(files.Label)], mean_mfccs)))

In [8]:
sp.apply(extract_MFCCfeatures, axis = 1)

0      None
1      None
2      None
3      None
4      None
       ... 
322    None
323    None
324    None
325    None
326    None
Length: 327, dtype: object

### 病理嗓音非线性特征参数（以下首先做小波包分解）

In [9]:
import pywt
from scipy.io import wavfile

In [10]:
def wavelet_packet_four_level(x):
    mother_wavelet = 'db10'
    wp = pywt.WaveletPacket(data = x, wavelet = mother_wavelet, mode = 'symmetric', maxlevel = 4)
    node_name_list = [node.path for node in wp.get_level(4, 'natural')]
    rec_results = []
    for i in node_name_list:
        new_wp = pywt.WaveletPacket(data=np.zeros(len(x)), wavelet=mother_wavelet, mode='symmetric')
        new_wp[i] = wp[i].data
        x_i = new_wp.reconstruct(update=True)
        rec_results.append(x_i)
    output = np.array(rec_results)
    return output

### Hurst

In [11]:
def Hurst(ts):
    ts = np.array(ts)
    N, RS, n = [], [], len(ts)
    while (True):
        N.append(n)
        m = np.mean(ts)
        mean_adj = ts - m
        cumulative_dvi = np.cumsum(mean_adj)
        srange = max(cumulative_dvi) - min(cumulative_dvi)
        unbiased_std_dvi = np.std(ts)
        RS.append(srange / unbiased_std_dvi)
        if n < 4:
            break
        ts, n = HalfSeries(ts, n)
    H = np.polyfit(np.log10(N), np.log10(RS), 1)[0]
    return H

def HalfSeries(s, n):
    X = []
    for i in range(0, len(s) - 1, 2):
        X.append((s[i] + s[i + 1]) / 2)
    # if length(s) is odd
    if len(s) % 2 != 0:
        X.append(s[-1])
        n = (n - 1) // 2
    else:
        n = n // 2
    return [np.array(X), n]

In [12]:
csvfile = open("feature_Hurst.csv", "w")
csvwriter = csv.writer(csvfile)
csvwriter.writerow(np.concatenate((['Label'], [i for i in range(1, 17)])))

46

In [13]:
def extract_Hurstfeatures(files):
    Fs, data = wavfile.read('./data/'+files.File)
    data = data/max(data)
    data_wp = wavelet_packet_four_level(data)
    feature = []
    for i in range(0,16):
        feature.append(Hurst(data_wp[i]))
    csvwriter.writerow(np.concatenate(([digit.index(files.Label)], feature)))

In [14]:
sp.apply(extract_Hurstfeatures, axis = 1)

0      None
1      None
2      None
3      None
4      None
       ... 
322    None
323    None
324    None
325    None
326    None
Length: 327, dtype: object

### Quadratic Renyi entropy

In [15]:
def calculate_renyi_entropy(sequence,alpha=2):
    hist, bin_edges = np.histogram(sequence, bins=100, density=True)
    prob = hist / hist.sum()
    prob_alpha = prob**alpha
    H = -np.log(prob_alpha.sum())
    return H/(alpha-1)

In [16]:
csvfile = open("feature_QR.csv", "w")
csvwriter = csv.writer(csvfile)
csvwriter.writerow(np.concatenate((['Label'], [i for i in range(1, 17)])))

46

In [17]:
def extract_QRfeatures(files):
    Fs, data = wavfile.read('./data/'+files.File)
    data = data/max(data)
    data_wp = wavelet_packet_four_level(data)
    feature = []
    for i in range(0,16):
        feature.append(calculate_renyi_entropy(data_wp[i]))
    csvwriter.writerow(np.concatenate(([digit.index(files.Label)], feature)))

In [18]:
sp.apply(extract_QRfeatures, axis = 1)

0      None
1      None
2      None
3      None
4      None
       ... 
322    None
323    None
324    None
325    None
326    None
Length: 327, dtype: object

### FMMI

In [19]:
# csvfile = open("feature_FMMI.csv", "w")
# csvwriter = csv.writer(csvfile)
# csvwriter.writerow(np.concatenate((['Label'], [i for i in range(1, 17)])))

In [20]:
def extract_FMMIfeatures(files):
    Fs, data = wavfile.read('./data/'+files.File)
    data = data/max(data)
    data_wp = wavelet_packet_four_level(data)
    feature = []
    for i in range(0,16):
        feature.append(fmmi(data_wp[i]))
    csvwriter.writerow(np.concatenate(([digit.index(files.Label)], feature)))

In [21]:
# sp.apply(extract_FMMIfeatures, axis = 1)

### Db

In [22]:
csvfile = open("feature_Db.csv", "w")
csvwriter = csv.writer(csvfile)
csvwriter.writerow(np.concatenate((['Label'], [i for i in range(1, 17)])))

46

In [23]:
import numpy as np

def box_count(seq, eps):
    """
    Calculates the box dimension of a continuous sequence using the box-counting method.
    :param seq: A numpy array of float numbers representing the continuous sequence.
    :param eps: The size of the boxes (epsilon) to use in the box-counting method.
    :return: The calculated box dimension.
    """
    N = len(seq)
    n_boxes = 0
    for eps in np.logspace(np.log10(eps), 0, 30):
        n_boxes = sum(np.any(np.abs(seq[i:i+int(1/eps)]) <= eps) for i in range(0,N,int(1/eps)))
        if n_boxes !=0:
            break

    return np.log(n_boxes) / np.log(1/eps)

In [24]:
def extract_Dbfeatures(files):
    Fs, data = wavfile.read('./data/'+files.File)
    data = data/max(data)
    data_wp = wavelet_packet_four_level(data)
    feature = []
    for i in range(0,16):
        feature.append(box_count(data_wp[i], 0.01))
    csvwriter.writerow(np.concatenate(([digit.index(files.Label)], feature)))

In [25]:
sp.apply(extract_Dbfeatures, axis = 1)

0      None
1      None
2      None
3      None
4      None
       ... 
322    None
323    None
324    None
325    None
326    None
Length: 327, dtype: object

### MFCC+Hurst

In [26]:
csvfile = open("feature_MFCCHurst.csv", "w")
csvwriter = csv.writer(csvfile)
csvwriter.writerow(np.concatenate((['Label'], [i for i in range(1, 30)])))

85

In [27]:
def extract_MFCCHurstfeatures(files):
    data, sr = librosa.load('./data/'+files.File)
    mean_mfccs = np.mean(librosa.feature.mfcc(y = data, sr = sr, n_mfcc = 13).T, axis = 0)
    
    Fs, data = wavfile.read('./data/'+files.File)
    data = data/max(data)
    data_wp = wavelet_packet_four_level(data)
    feature = []
    for i in range(0,16):
        feature.append(Hurst(data_wp[i]))
    csvwriter.writerow(np.concatenate(([digit.index(files.Label)], mean_mfccs, feature)))

In [28]:
sp.apply(extract_MFCCHurstfeatures, axis = 1)

0      None
1      None
2      None
3      None
4      None
       ... 
322    None
323    None
324    None
325    None
326    None
Length: 327, dtype: object

### MFCC+QR

In [29]:
csvfile = open("feature_MFCCQR.csv", "w")
csvwriter = csv.writer(csvfile)
csvwriter.writerow(np.concatenate((['Label'], [i for i in range(1, 30)])))

85

In [30]:
def extract_MFCCQRfeatures(files):
    data, sr = librosa.load('./data/'+files.File)
    mean_mfccs = np.mean(librosa.feature.mfcc(y = data, sr = sr, n_mfcc = 13).T, axis = 0)
    
    Fs, data = wavfile.read('./data/'+files.File)
    data = data/max(data)
    data_wp = wavelet_packet_four_level(data)
    feature = []
    for i in range(0,16):
        feature.append(calculate_renyi_entropy(data_wp[i]))
    csvwriter.writerow(np.concatenate(([digit.index(files.Label)], mean_mfccs, feature)))

In [31]:
sp.apply(extract_MFCCQRfeatures, axis = 1)

0      None
1      None
2      None
3      None
4      None
       ... 
322    None
323    None
324    None
325    None
326    None
Length: 327, dtype: object

### Hurst+QR

In [32]:
csvfile = open("feature_HurstQR.csv", "w")
csvwriter = csv.writer(csvfile)
csvwriter.writerow(np.concatenate((['Label'], [i for i in range(1, 33)])))

94

In [33]:
def extract_HurstQRfeatures(files):
    Fs, data = wavfile.read('./data/'+files.File)
    data = data/max(data)
    data_wp = wavelet_packet_four_level(data)
    feature = []
    for i in range(0,16):
        feature.append(Hurst(data_wp[i]))
    for i in range(0,16):
        feature.append(calculate_renyi_entropy(data_wp[i]))
    csvwriter.writerow(np.concatenate(([digit.index(files.Label)], feature)))

In [34]:
sp.apply(extract_HurstQRfeatures, axis = 1)

0      None
1      None
2      None
3      None
4      None
       ... 
322    None
323    None
324    None
325    None
326    None
Length: 327, dtype: object

### MFCC+Hurst+QR

In [35]:
csvfile = open("feature_MFCCHurstQR.csv", "w")
csvwriter = csv.writer(csvfile)
csvwriter.writerow(np.concatenate((['Label'], [i for i in range(1, 46)])))

133

In [36]:
def extract_MFCCHurstQRfeatures(files):
    data, sr = librosa.load('./data/'+files.File)
    mean_mfccs = np.mean(librosa.feature.mfcc(y = data, sr = sr, n_mfcc = 13).T, axis = 0)
    
    Fs, data = wavfile.read('./data/'+files.File)
    data = data/max(data)
    data_wp = wavelet_packet_four_level(data)
    feature = []
    for i in range(0,16):
        feature.append(Hurst(data_wp[i]))
    for i in range(0,16):
        feature.append(calculate_renyi_entropy(data_wp[i]))
    csvwriter.writerow(np.concatenate(([digit.index(files.Label)], mean_mfccs, feature)))

In [37]:
sp.apply(extract_MFCCHurstQRfeatures, axis = 1)

0      None
1      None
2      None
3      None
4      None
       ... 
322    None
323    None
324    None
325    None
326    None
Length: 327, dtype: object

### MFCC+Hurst+QR+Db

In [38]:
csvfile = open("feature_MFCCHurstQRDb.csv", "w")
csvwriter = csv.writer(csvfile)
csvwriter.writerow(np.concatenate((['Label'], [i for i in range(1, 62)])))

181

In [39]:
def extract_MFCCHurstQRDbfeatures(files):
    data, sr = librosa.load('./data/'+files.File)
    mean_mfccs = np.mean(librosa.feature.mfcc(y = data, sr = sr, n_mfcc = 13).T, axis = 0)
    
    Fs, data = wavfile.read('./data/'+files.File)
    data = data/max(data)
    data_wp = wavelet_packet_four_level(data)
    feature = []
    for i in range(0,16):
        feature.append(Hurst(data_wp[i]))
    for i in range(0,16):
        feature.append(calculate_renyi_entropy(data_wp[i]))
    for i in range(0,16):
        feature.append(box_count(data_wp[i], 0.01))
    csvwriter.writerow(np.concatenate(([digit.index(files.Label)], mean_mfccs, feature)))

In [40]:
sp.apply(extract_MFCCHurstQRDbfeatures, axis = 1)

0      None
1      None
2      None
3      None
4      None
       ... 
322    None
323    None
324    None
325    None
326    None
Length: 327, dtype: object

### MFCC+Hurst+Db

In [41]:
csvfile = open("feature_MFCCHurstDb.csv", "w")
csvwriter = csv.writer(csvfile)
csvwriter.writerow(np.concatenate((['Label'], [i for i in range(1, 46)])))

133

In [42]:
def extract_MFCCHurstDbfeatures(files):
    data, sr = librosa.load('./data/'+files.File)
    mean_mfccs = np.mean(librosa.feature.mfcc(y = data, sr = sr, n_mfcc = 13).T, axis = 0)
    
    Fs, data = wavfile.read('./data/'+files.File)
    data = data/max(data)
    data_wp = wavelet_packet_four_level(data)
    feature = []
    for i in range(0,16):
        feature.append(Hurst(data_wp[i]))
    for i in range(0,16):
        feature.append(box_count(data_wp[i], 0.01))
    csvwriter.writerow(np.concatenate(([digit.index(files.Label)], mean_mfccs, feature)))

In [43]:
sp.apply(extract_MFCCHurstDbfeatures, axis = 1)

0      None
1      None
2      None
3      None
4      None
       ... 
322    None
323    None
324    None
325    None
326    None
Length: 327, dtype: object

### MFCC+QR+Db

In [44]:
csvfile = open("feature_MFCCQRDb.csv", "w")
csvwriter = csv.writer(csvfile)
csvwriter.writerow(np.concatenate((['Label'], [i for i in range(1, 46)])))

133

In [45]:
def extract_MFCCQRDbfeatures(files):
    data, sr = librosa.load('./data/'+files.File)
    mean_mfccs = np.mean(librosa.feature.mfcc(y = data, sr = sr, n_mfcc = 13).T, axis = 0)
    
    Fs, data = wavfile.read('./data/'+files.File)
    data = data/max(data)
    data_wp = wavelet_packet_four_level(data)
    feature = []
    for i in range(0,16):
        feature.append(calculate_renyi_entropy(data_wp[i]))
    for i in range(0,16):
        feature.append(box_count(data_wp[i], 0.01))
    csvwriter.writerow(np.concatenate(([digit.index(files.Label)], mean_mfccs, feature)))

In [46]:
sp.apply(extract_MFCCQRDbfeatures, axis = 1)

0      None
1      None
2      None
3      None
4      None
       ... 
322    None
323    None
324    None
325    None
326    None
Length: 327, dtype: object